In [ ]:
from typing import List, Dict, Tuple
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd

In [ ]:
# model_name, model_type = "sberbank-ai/ruT5-base", "pytorch"
model_name, model_type = "naltukhov/joke-generator-t5-rus-finetune", "flax"
tokenizer = T5Tokenizer.from_pretrained(model_name, from_flax=model_type == "flax",
                                        force_download=True, use_auth_token=True)
model = T5ForConditionalGeneration.from_pretrained(model_name, from_flax=model_type == "flax",
                                                   force_download=True, use_auth_token=True)
print(f'Loaded model {model_name}')

inspiration_prefix = 'Сгенерировать вдохновение: '
mark_prefix = 'Сгенерировать оценку: '
punch_prefix = 'Сгенерировать шутку: '

In [ ]:
def generate_inspirations(setup, num_return_sequences=5, temperature=1):
    
    # Generate inspirations
    setup_ids = tokenizer(inspiration_prefix + setup, return_tensors="pt").input_ids
    predict_inspiration_ids = model.generate(setup_ids,
                                             top_k=20,
                                             do_sample=True,
                                             max_length=50,
                                             no_repeat_ngram_size=2,
                                             temperature=temperature,
                                             num_return_sequences=num_return_sequences).tolist()
    predict_inspirations = [tokenizer.decode(p, skip_special_tokens=True) for p in predict_inspiration_ids]
    return predict_inspirations

In [ ]:
def generate_punches(setup, inspiration, num_return_sequences=5, temperature=1):
    input_ids = tokenizer(punch_prefix + inspiration + '|' + setup, return_tensors="pt").input_ids
    predict_punches_ids = model.generate(input_ids,
                                         do_sample=True,
                                         top_k=20,
                                         max_length=50,
                                         no_repeat_ngram_size=2,
                                         temperature=temperature,
                                         num_return_sequences=num_return_sequences).tolist()
    predict_punches = [tokenizer.decode(p, skip_special_tokens=True) for p in predict_punches_ids]
    return predict_punches

In [ ]:
def generate_mark(joke):
    input_ids = tokenizer(mark_prefix + joke, return_tensors="pt").input_ids
    predict_mark_ids = model.generate(input_ids).tolist()
    predict_mark = tokenizer.decode(predict_mark_ids[0], skip_special_tokens=True)
    return predict_mark

In [ ]:
def inference(setup, inspirations: List=None, 
              num_return_sequences: int=5, 
              temperature: int=1) -> List[Tuple[str, str, str, str]]:
    result_list = list()
    if not inspirations:
        # Generate inspirations
        inspirations = generate_inspirations(setup, 
                                             num_return_sequences=num_return_sequences,
                                             temperature=temperature,)
    for inspiration in inspirations:
        # Generate punches
        punches = generate_punches(setup, 
                                   inspiration, 
                                   num_return_sequences=num_return_sequences,
                                   temperature=temperature)
        for punch in punches:
            joke = setup + punch
            mark = generate_mark(joke)
            result_list.append((setup, inspiration, punch, mark))
    return result_list

In [ ]:
setup = 'Российские войска ведут штурм и наступают на двух направлениях'
setup = 'Медведев приехал на церемонию прощания с Горбачевым'
# setup = 'Я плохой рассказчик'
# setup = 'Медведь шёл по лесу'

predicts = inference(setup, 
                     num_return_sequences=3, 
                     temperature=0.7)

In [ ]:
predicts_df = pd.DataFrame(predicts, columns=['setup', 'inspiration', 'punch', 'mark'])
predicts_df.to_dict(orient='records')

In [51]:
def generate_inspirations(setup, num_return_sequences=5, temperature=1):
    
    # Generate inspirations
    setup_ids = tokenizer(inspiration_prefix + setup, return_tensors="pt").input_ids
    predict_inspiration_ids = model.generate(setup_ids,
                                             top_k=20,
                                             do_sample=True,
                                             max_length=50,
                                             no_repeat_ngram_size=2,
                                             temperature=temperature,
                                             num_return_sequences=num_return_sequences).tolist()
    predict_inspirations = [tokenizer.decode(p, skip_special_tokens=True) for p in predict_inspiration_ids]
    return predict_inspirations

In [52]:
def generate_punches(setup, inspiration, num_return_sequences=5, temperature=1):
    input_ids = tokenizer(punch_prefix + inspiration + '|' + setup, return_tensors="pt").input_ids
    predict_punches_ids = model.generate(input_ids,
                                         do_sample=True,
                                         top_k=20,
                                         max_length=50,
                                         no_repeat_ngram_size=2,
                                         temperature=temperature,
                                         num_return_sequences=num_return_sequences).tolist()
    predict_punches = [tokenizer.decode(p, skip_special_tokens=True) for p in predict_punches_ids]
    return predict_punches

In [53]:
def generate_mark(joke):
    input_ids = tokenizer(mark_prefix + joke, return_tensors="pt").input_ids
    predict_mark_ids = model.generate(input_ids).tolist()
    predict_mark = tokenizer.decode(predict_mark_ids[0], skip_special_tokens=True)
    return predict_mark

In [60]:
def inference(setup, inspirations: List=None, 
              num_return_sequences: int=5, 
              temperature: int=1) -> List[Tuple[str, str, str, str]]:
    result_list = list()
    if not inspirations:
        # Generate inspirations
        inspirations = generate_inspirations(setup, 
                                             num_return_sequences=num_return_sequences,
                                             temperature=temperature,)
    for inspiration in inspirations:
        # Generate punches
        punches = generate_punches(setup, 
                                   inspiration, 
                                   num_return_sequences=num_return_sequences,
                                   temperature=temperature)
        for punch in punches:
            joke = setup + punch
            mark = generate_mark(joke)
            result_list.append((setup, inspiration, punch, mark))
    return result_list

In [75]:
setup = 'Российские войска ведут штурм и наступают на двух направлениях'
setup = 'Медведев приехал на церемонию прощания с Горбачевым'
# setup = 'Я плохой рассказчик'
# setup = 'Медведь шёл по лесу'

predicts = inference(setup, 
                     num_return_sequences=3, 
                     temperature=0.7)

In [76]:
predicts_df = pd.DataFrame(predicts, columns=['setup', 'inspiration', 'punch', 'mark'])
predicts_df.to_dict(orient='records')

[{'setup': 'Медведев приехал на церемонию прощания с Горбачевым',
  'inspiration': 'деньга натёр',
  'punch': 'Он узнал, что деньги натёр и не погасился',
  'mark': 'плохо'},
 {'setup': 'Медведев приехал на церемонию прощания с Горбачевым',
  'inspiration': 'деньга натёр',
  'punch': 'Деньги натёр он',
  'mark': 'плохо'},
 {'setup': 'Медведев приехал на церемонию прощания с Горбачевым',
  'inspiration': 'деньга натёр',
  'punch': 'Деньги натёр он и пырнулнул в экстазе',
  'mark': 'плохо'},
 {'setup': 'Медведев приехал на церемонию прощания с Горбачевым',
  'inspiration': 'поблагодарить коллега',
  'punch': 'А он поблагодарил коллегу за поддержку',
  'mark': 'плохо'},
 {'setup': 'Медведев приехал на церемонию прощания с Горбачевым',
  'inspiration': 'поблагодарить коллега',
  'punch': 'Он поблагодарил коллегу за поздравления',
  'mark': 'плохо'},
 {'setup': 'Медведев приехал на церемонию прощания с Горбачевым',
  'inspiration': 'поблагодарить коллега',
  'punch': 'В благодарность коллег

In [ ]:
# Punch generation
df = pd.read_csv('data/agg-generation-dataset/agg-generation-dataset-test.csv')
df = df.loc[df.input.str.startswith(punch_prefix)]
for i in range(10):
    input, output = df.sample().values[0]
    print(f'\tExample {i + 1}:')

    predicts = inference(input)
    print(*predicts)
